## __1. Import Packages__

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score 
import pickle
import os
import csv

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

## __2. Video Capture Code__

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while(cap.isOpened()):
        success, image = cap.read()
            
        if not success:
            print("Empty camera frame!")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # converts webcam image to opencv bgr image
        results = holistic.process(image)
            
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
            
        # draw face landmarks 
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # right hand 
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # pose detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        cv2.imshow('Camera', image) # display the frame

        k = cv2.waitKey(30) & 0xff
        if k == 27: # press 'esc' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

## __3. Get Training Data__
### How to Get Data
- Set the class_name to the pose name and score, i.e ```HeadSittingScore3```
- Use video capture code and extract desired landmarks
- Save landmarks to ```data.csv```
- Get into position of said pose name and score, following the HINE evaluation model

In [ ]:
class_name = "HeadSittingScore3"

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while(cap.isOpened()):
        success, image = cap.read()
            
        if not success:
            print("Empty camera frame!")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # converts webcam image to opencv bgr image
        results = holistic.process(image)
            
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
            
        # draw face landmarks 
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # right hand 
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # pose detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        try:
            # extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # concate rows
            row = pose_row+face_row
            
            # append class name 
            row.insert(0, class_name)

            # export to CSV
            with open('data.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass

        cv2.imshow('Camera', image) # display the frame

        k = cv2.waitKey(30) & 0xff
        if k == 27: # press 'esc' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

## __4. Training Model with Scikit Learn__
### Read and Process Data

In [ ]:
data = pd.read_csv('data.csv') # open the csv

In [ ]:
X = data.drop('class', axis = 1) # features
y = data['class'] # class to predict

In [ ]:
# split the data into 2 data sets, train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

### Train the Classification Model

In [ ]:
# 4 different types of learning algorithms to test
pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=10000)),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
# test each learning algorithm by fitting the data to each
# .values to extract data from the dataframe prior to fitting
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train.values, y_train.values)
    fit_models[algo] = model

In [ ]:
fit_models

### Determine Which Model to Use

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test.values) # test the model on X_test
    print(algo, accuracy_score(y_test, yhat)) # compare our model's prediction againt the y_test, display the accuracy 

In [ ]:
fit_models['rc'].predict(X_test.values) 

In [ ]:
y_test

In [ ]:
# ridge classification was the most accurate, serialize and save the model via pickling
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rc'], f)

## __5. Make Detections__
### How to Use the Model
- Load the model 
- Use video capture code and extract landmarks
- Pass the landmarks as a np array to the model
- Display prediction on the screen

In [ ]:
# deserialize the model and load it to use
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
model

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while(cap.isOpened()):
        success, image = cap.read()
            
        if not success:
            print("Empty camera frame!")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
            
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # converts webcam image to opencv bgr image
            
        # draw face landmarks 
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # right hand 
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # pose detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        try:
            # extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # concate rows
            row = pose_row+face_row

            X = pd.DataFrame([row]) # convert row to a dataframe

            body_language_class = model.predict(X.values)[0]
            body_language_prob = model.predict_proba(X.values)[0]
            # print(body_language_class, body_language_prob)
            
            # display probability
            cv2.putText(image, 'probability:'
                        , (15,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (105,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)


            # display class
            cv2.putText(image, 'class:'
                        , (15,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (65,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        except:
            pass

        cv2.imshow('Camera', image) # display the frame

        k = cv2.waitKey(30) & 0xff
        if k == 27: # press 'esc' to quit
            break

    cap.release()
    cv2.destroyAllWindows()